In [2]:
import os 

import numpy as np 
import pandas as pd  
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import nbimporter
import Useful_Visualization_Functions
from pyspark.ml import *
from pyspark.sql import *
from pyspark.ml.classification import LinearSVC
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import Imputer, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.types import *
from pyspark.sql.functions import *
# from pyspark.sql.functions import col, explode, array, lit, lead, when, substring
warnings.filterwarnings("ignore")
import pyspark.sql.functions as F
from pyspark.sql import Window

In [3]:
# from pyspark.sql import SparkSession
# from pyspark.sql import Row
# from pyspark.sql.functions import lit, col, column, expr, desc, asc

In [4]:
# ! pip install matplotlib
# ! pip install seaborn
# ! pip install ipynb
# ! pip install nbimporter

In [20]:
# build our own SparkSession
myspark = SparkSession\
    .builder\
    .appName("AWS-Spark")\
    .config("spark.driver.memory", "12g") \
    .config("spark.sql.shuffle.partitions",6)\
    .config("spark.sql.repl.eagereval.enabled",True)\
    .getOrCreate()

ConnectionRefusedError: [Errno 111] Connection refused

In [6]:
myspark

In [7]:
# ! head noaa.csv
# noaa_data.show(10)
#noaa_data = myspark.read.load("noaa.csv", format="csv", sep=",", header=True, inferSchema=True)
noaa_csv_pathname = "noaa.csv"

noaa_data = myspark.read.load(noaa_csv_pathname, format="csv", sep=",", header=True, inferSchema=True)

noaa_data.count()

25933550

In [8]:
# noaa_data.show(10)

In [9]:
# temp_filt = (noaa_data.filter(noaa_data.ELEVATION <= 5))
# temp_filt.count()

In [10]:
#latitude_order = noaa_data.orderBy("LATITUDE", ascending=False)
#latitude_order.show(10)

In [11]:
noaa_data.printSchema()

root
 |-- STATION: string (nullable = true)
 |-- DATE: string (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- ELEVATION: double (nullable = true)
 |-- NAME: string (nullable = true)
 |-- TEMP: double (nullable = true)
 |-- TEMP_ATTRIBUTES: double (nullable = true)
 |-- DEWP: double (nullable = true)
 |-- DEWP_ATTRIBUTES: double (nullable = true)
 |-- SLP: double (nullable = true)
 |-- SLP_ATTRIBUTES: double (nullable = true)
 |-- STP: double (nullable = true)
 |-- STP_ATTRIBUTES: double (nullable = true)
 |-- VISIB: double (nullable = true)
 |-- VISIB_ATTRIBUTES: double (nullable = true)
 |-- WDSP: double (nullable = true)
 |-- WDSP_ATTRIBUTES: double (nullable = true)
 |-- MXSPD: double (nullable = true)
 |-- GUST: double (nullable = true)
 |-- MAX: double (nullable = true)
 |-- MAX_ATTRIBUTES: string (nullable = true)
 |-- MIN: double (nullable = true)
 |-- MIN_ATTRIBUTES: string (nullable = true)
 |-- PRCP: double (nullable = t

In [12]:
noaa_data.select("TEMP", "ELEVATION", "VISIB").show(10)

+----+---------+-----+
|TEMP|ELEVATION|VISIB|
+----+---------+-----+
|22.5|      9.0|  9.8|
|21.0|      9.0| 14.0|
|21.6|      9.0| 11.9|
|19.5|      9.0|  7.3|
|11.4|      9.0|  0.7|
|12.8|      9.0|  4.8|
|12.1|      9.0|  3.9|
|25.8|      9.0|  6.2|
|29.9|      9.0|  4.4|
|35.8|      9.0|  5.4|
+----+---------+-----+
only showing top 10 rows



In [13]:
noaa_data = noaa_data.withColumn("ItRained", when((F.length(noaa_data["FRSHTT"]) <= 4), lit(0)) \
                    .when(F.length(noaa_data["FRSHTT"]) == 5, lit(1)) \
                    .otherwise(lit(substring('FRSHTT', 2, 1).cast(IntegerType()))) \
)

#noaa_data.show(10)
noaa_data = noaa_data.withColumn( "NextDay", lead("ItRained", default=2).over(Window.orderBy("STATION")).alias("Next") )
print(noaa_data.count())

25933550


In [14]:
columns = noaa_data.columns
"""
for cl in columns:
    noaa_data.describe(cl).show()

for cl in columns:
    noaa_data.select(cl).distinct().show(10)
"""

'\nfor cl in columns:\n    noaa_data.describe(cl).show()\n\nfor cl in columns:\n    noaa_data.select(cl).distinct().show(10)\n'

In [15]:
cols_to_drop = ["STATION", "DATE", "LATITUDE", "LONGITUDE", "ELEVATION", "NAME", "TEMP_ATTRIBUTES", "DEWP_ATTRIBUTES",
               "SLP_ATTRIBUTES", "STP_ATTRIBUTES", "VISIB_ATTRIBUTES", "WDSP_ATTRIBUTES", "MAX_ATTRIBUTES",
               "MIN_ATTRIBUTES", "PRCP_ATTRIBUTES", "GUST", "ItRained"]

cols_interest = [x for x in columns if x not in cols_to_drop]
df_interest_cols = noaa_data.select(cols_interest)

### Data cleansing

In [16]:
df_interest_cols.printSchema()
df_clean = df_interest_cols.dropna()
[df_interest_cols.count(), df_clean.count()]

columns = df_clean.columns

"""
for cl in df_clean.columns: 
    print(cl)
    df_clean.select(cl).summary().show()
df_clean.select("ItRained").summary().show()

for cl in columns:
    df_clean.describe(cl).show()


for cl in columns:
    df_clean.select(cl).distinct().show(10)
"""


root
 |-- TEMP: double (nullable = true)
 |-- DEWP: double (nullable = true)
 |-- SLP: double (nullable = true)
 |-- STP: double (nullable = true)
 |-- VISIB: double (nullable = true)
 |-- WDSP: double (nullable = true)
 |-- MXSPD: double (nullable = true)
 |-- MAX: double (nullable = true)
 |-- MIN: double (nullable = true)
 |-- PRCP: double (nullable = true)
 |-- SNDP: double (nullable = true)
 |-- FRSHTT: integer (nullable = true)
 |-- NextDay: integer (nullable = true)



22/05/24 11:00:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/24 11:00:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


'\nfor cl in df_clean.columns: \n    print(cl)\n    df_clean.select(cl).summary().show()\ndf_clean.select("ItRained").summary().show()\n\nfor cl in columns:\n    df_clean.describe(cl).show()\n\n\nfor cl in columns:\n    df_clean.select(cl).distinct().show(10)\n'

In [17]:
print(df_clean.count())

df_clean = df_clean.filter(df_clean.TEMP > -10)
df_clean = df_clean.filter(df_clean.DEWP < 100)
df_clean = df_clean.filter(df_clean.SLP < 4000)
df_clean = df_clean.filter(df_clean.STP < 100)
df_clean = df_clean.filter(df_clean.VISIB < 100)
df_clean = df_clean.filter(df_clean.WDSP < 100)
df_clean = df_clean.filter(df_clean.MXSPD < 100)
# df_clean = df_clean.filter(df_clean.GUST < 100)
df_clean = df_clean.filter(df_clean.MAX < 100)
df_clean = df_clean.filter(df_clean.MIN < 100)
df_clean = df_clean.filter(df_clean.PRCP < 100)
df_clean = df_clean.filter(df_clean.SNDP < 100)


"""
temp_median = df_clean_pd['TEMP'].quantile(0.50)
df_clean_pd['TEMP'] = np.where(df_clean_pd['TEMP'] < -10, temp_median, df_clean_pd['TEMP'])
plt.boxplot(df_clean_pd["TEMP"])
plt.show()

dewp_median = df_clean_pd['DEWP'].quantile(0.50)
df_clean_pd['DEWP'] = np.where(df_clean_pd['DEWP'] > 100, dewp_median, df_clean_pd['DEWP'])
plt.boxplot(df_clean_pd["DEWP"])
plt.show()
"""

22/05/24 11:01:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/24 11:01:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


25933550


'\ntemp_median = df_clean_pd[\'TEMP\'].quantile(0.50)\ndf_clean_pd[\'TEMP\'] = np.where(df_clean_pd[\'TEMP\'] < -10, temp_median, df_clean_pd[\'TEMP\'])\nplt.boxplot(df_clean_pd["TEMP"])\nplt.show()\n\ndewp_median = df_clean_pd[\'DEWP\'].quantile(0.50)\ndf_clean_pd[\'DEWP\'] = np.where(df_clean_pd[\'DEWP\'] > 100, dewp_median, df_clean_pd[\'DEWP\'])\nplt.boxplot(df_clean_pd["DEWP"])\nplt.show()\n'

### Undersampling

In [18]:

print(df_clean.count())
zero_df = df_clean.filter(col("NextDay") == 0)
one_df = df_clean.filter(col("NextDay") == 1)

# major_df, minor_df = zero_df, one_df if zero_df.count() > one_df.count() else one_df, zero_df

if zero_df.count() > one_df.count():
    major_df = zero_df
    minor_df = one_df
else:
    major_df = one_df
    minor_df = zero_df
    
ratio = major_df.count()/minor_df.count()
sampled_majority_df = major_df.sample(False, 1/ratio)
df_clean = sampled_majority_df.unionAll(minor_df)
df_clean.select("NextDay").summary().show()

22/05/24 11:01:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/24 11:01:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/24 11:02:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


415129


22/05/24 11:02:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/24 11:02:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/24 11:02:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/24 11:03:14 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/24 11:03:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/24 11:03:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/24 1

+-------+------------------+
|summary|           NextDay|
+-------+------------------+
|  count|            115857|
|   mean|0.5219278938691663|
| stddev|0.4995210918182815|
|    min|                 0|
|    25%|                 0|
|    50%|                 1|
|    75%|                 1|
|    max|                 1|
+-------+------------------+



In [19]:
cleanfilename = "clean-noaa"
df_clean.write.mode("overwrite").parquet(cleanfilename)

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/nuno/.local/lib/python3.8/site-packages/py4j/clientserver.py", line 480, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/nuno/.local/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/nuno/.local/lib/python3.8/site-packages/py4j/clientserver.py", line 503, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/nuno/.local/lib/python3.8/site-packages/py4j/clientserver.py", line 480, in send_command
    raise Py4JNetworkError("Answer from Java side is empty"

Py4JError: An error occurred while calling o161.write

In [55]:
# df_clean = myspark.createDataFrame(df_clean_pd)
#df_train, df_test = df_clean.randomSplit([0.8,0.2], seed = 42)
#df_train.cache()
#print(f"There are {df_train.count()} rows in the training set and {df_test.count()} in the test set")

df_train_n, df_test_n = df_clean_n.randomSplit([0.8,0.2], seed = 42)
df_train_n.cache()
print(f"There are {df_train_n.count()} rows in the training set and {df_test_n.count()} in the test set")

22/05/23 15:29:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/23 15:29:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/23 15:29:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/23 15:29:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/23 15:29:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/23 15:29:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


60783


22/05/23 15:29:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/23 15:29:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


15055


22/05/23 15:29:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/23 15:29:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/23 15:30:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/23 15:30:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/23 15:30:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/23 15:30:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


There are 60783 rows in the training set and 15055 in the test set


In [56]:
#vec_assembler = VectorAssembler(inputCols=['TEMP', 'DEWP','SLP', 'STP', 'VISIB', 'WDSP', 'MXSPD', 'MAX', 'MIN'], outputCol="features")
#vec_df_train = vec_assembler.transform(df_train)
#
## show the content of the columns bedrooms, features and price
## vec_df_train.select("TEMP","DEWP","features").show(200)
#
#lsvc = LinearSVC(maxIter=10, regParam=0.1, labelCol="ItRained")
#pipeline = Pipeline(stages=[vec_assembler, lsvc])
#pipeline_model = pipeline.fit(df_train)
#df_prediction = pipeline_model.transform(df_test)
## df_prediction.select("features", "ItRained", "prediction").sort("prediction", ascending=False).show(200)
#
#prediction_label = df_prediction.select("prediction", "ItRained")  
#
## supports metricName="areaUnderROC" (default) and "areaUnderPR"
## it relates sensitivity (TP rate) and specificity (FP rate)
#
#evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='ItRained', )
#
## print("areaUnderROC = " + str(evaluator.evaluate(prediction_label)))
#
#n = df_prediction.count()
#tp = df_prediction.filter(expr("prediction > 0") & expr("ItRained == prediction")).count()
#tn = df_prediction.filter(expr("prediction <= 0") & expr("ItRained == prediction")).count()
#fp = df_prediction.filter(expr("prediction > 0") & expr("ItRained != prediction")).count()
#fn = n - tp - tn - fp
#print("True Positive: ",tp/n * 100, 2,"%", "\nTrue Negative: ", tn/n * 100,"%",
#      "\nFalse Positive: ", fp/n * 100 ,"%", "\nFalse Negative: ", fn/n * 100,"%", 
#      "\nPrediction count:", n)


#====================================================================================================================================================

vec_assembler_n = VectorAssembler(inputCols=['TEMP', 'DEWP','SLP', 'STP', 'VISIB', 'WDSP', 'MXSPD', 'MAX', 'MIN'], outputCol="features")
vec_df_train_n = vec_assembler_n.transform(df_train_n)

# show the content of the columns bedrooms, features and price
# vec_df_train.select("TEMP","DEWP","features").show(200)

lsvc_n = LinearSVC(maxIter=10, regParam=0.1, labelCol="NextDay")
pipeline_n = Pipeline(stages=[vec_assembler_n, lsvc_n])
pipeline_model_n = pipeline_n.fit(df_train_n)
df_prediction_n = pipeline_model_n.transform(df_test_n)
# df_prediction.select("features", "ItRained", "prediction").sort("prediction", ascending=False).show(200)

prediction_label_n = df_prediction_n.select("prediction", "NextDay")  

# supports metricName="areaUnderROC" (default) and "areaUnderPR"
# it relates sensitivity (TP rate) and specificity (FP rate)

evaluator_n = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='NextDay', )

# print("areaUnderROC = " + str(evaluator.evaluate(prediction_label)))

n_n = df_prediction_n.count()
tp_n = df_prediction_n.filter(expr("prediction > 0") & expr("NextDay == prediction")).count()
tn_n = df_prediction_n.filter(expr("prediction <= 0") & expr("NextDay == prediction")).count()
fp_n = df_prediction_n.filter(expr("prediction > 0") & expr("NextDay != prediction")).count()
fn_n = n_n - tp_n - tn_n - fp_n
print("True Positive: ",tp_n/n_n * 100, 2,"%", "\nTrue Negative: ", tn_n/n_n * 100,"%",
      "\nFalse Positive: ", fp_n/n_n * 100 ,"%", "\nFalse Negative: ", fn_n/n_n * 100,"%", 
      "\nPrediction count:", n_n)


"""
prediction_label = df_prediction.select("prediction", "ItRained")

evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction')

print("areaUnderROC = " + str(evaluator.evaluate(prediction_label)))

lr = LinearRegression(featuresCol="features", labelCol="ItRained")
lr_model = lr.fit(vec_df_train)
avg_ItRained = float(df_train.select(avg("ItRained")).first()[0])
df_pred = df_train.withColumn("avg_ItRained_prediction", lit(avg_ItRained))
avg_ItRained
evaluator = RegressionEvaluator(predictionCol="avg_ItRained_prediction", labelCol="ItRained", metricName="rmse")
print(f"The RMSE for predicting the average frshtt is: {evaluator.evaluate(df_pred):.2f}")
pipeline = Pipeline(stages=[vec_assembler, lr_model])

# get the model (as transformer)
pipeline_model = pipeline.fit(df_train)
df_prediction = pipeline_model.transform(df_test)

# show the columns worth to be looked at
df_prediction.select("features","ItRained","prediction").sample(False, 0.1).sort("ItRained", ascending=False).show(200)

df_prediction.columns
"""

22/05/23 15:30:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/23 15:30:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/23 15:30:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/23 15:30:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/23 15:31:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/23 15:31:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/23 1

True Positive:  41.806708734639656 2 % 
True Negative:  31.8365991364995 % 
False Positive:  18.432414480239125 % 
False Negative:  7.924277648621721 % 
Prediction count: 15055


'\nprediction_label = df_prediction.select("prediction", "ItRained")\n\nevaluator = BinaryClassificationEvaluator(rawPredictionCol=\'prediction\')\n\nprint("areaUnderROC = " + str(evaluator.evaluate(prediction_label)))\n\nlr = LinearRegression(featuresCol="features", labelCol="ItRained")\nlr_model = lr.fit(vec_df_train)\navg_ItRained = float(df_train.select(avg("ItRained")).first()[0])\ndf_pred = df_train.withColumn("avg_ItRained_prediction", lit(avg_ItRained))\navg_ItRained\nevaluator = RegressionEvaluator(predictionCol="avg_ItRained_prediction", labelCol="ItRained", metricName="rmse")\nprint(f"The RMSE for predicting the average frshtt is: {evaluator.evaluate(df_pred):.2f}")\npipeline = Pipeline(stages=[vec_assembler, lr_model])\n\n# get the model (as transformer)\npipeline_model = pipeline.fit(df_train)\ndf_prediction = pipeline_model.transform(df_test)\n\n# show the columns worth to be looked at\ndf_prediction.select("features","ItRained","prediction").sample(False, 0.1).sort("ItRa